In [ ]:
import os
import time
import logging
from paddleocr import PaddleOCR
from dataclasses import dataclass

# --- Configuration ---
@dataclass
class Config:
    IMAGE_FOLDER = r"D:\New folder (2)"   # Input folder
    OUTPUT_FOLDER = r"D:\ocr_new_files"         # Output folder for OCR text
    SUPPORTED_EXTENSIONS = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff')

config = Config()

# --- Logging setup ---
os.makedirs(config.OUTPUT_FOLDER, exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(config.OUTPUT_FOLDER, 'ocr_run.log')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# --- Initialize PaddleOCR ---
print("Initializing PaddleOCR reader...")
try:
    ocr_reader = PaddleOCR(lang='en')
    print("✅ PaddleOCR reader initialized successfully.")
    logger.info("PaddleOCR initialized successfully.")
except Exception as e:
    print(f"❌ Error initializing PaddleOCR: {e}")
    logger.error(f"Error initializing PaddleOCR: {e}")
    exit()

# --- OCR extraction function ---
class SimpleLineOCRExtractor:
    """Performs line-structured OCR extraction using PaddleOCR"""
    @staticmethod
    def extract_with_line_structure(image_path: str):
        start_time = time.time()
        try:
            result = ocr_reader.predict(image_path)
            extracted_texts = []
            if result and result[0] and 'rec_texts' in result[0]:
                extracted_texts = result[0]['rec_texts']
            structured_text = "\n".join([text.strip() for text in extracted_texts if text.strip()])
            latency = time.time() - start_time
            return structured_text, latency
        except Exception as e:
            logger.error(f"OCR failed for {image_path}: {e}")
            return "", time.time() - start_time

# --- Save OCR text (without extraction details) ---
def perform_ocr_and_save(image_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    extracted_text, processing_time = SimpleLineOCRExtractor.extract_with_line_structure(image_path)

    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_text_file = os.path.join(output_dir, f"{base_name}_ocr.txt")

    # ✅ Save ONLY the extracted text (no details at the bottom)
    with open(output_text_file, "w", encoding="utf-8") as f:
        f.write(extracted_text)

    return extracted_text, processing_time

# --- Main driver function ---
def main():
    if not os.path.isdir(config.IMAGE_FOLDER):
        print(f"❌ Image folder not found: {config.IMAGE_FOLDER}")
        return

    image_files = [
        f for f in os.listdir(config.IMAGE_FOLDER)
        if f.lower().endswith(config.SUPPORTED_EXTENSIONS)
    ]
    if not image_files:
        print("⚠️ No image files found.")
        return

    print(f"🖼️ Found {len(image_files)} images to process.\n")

    for image_name in image_files:
        image_path = os.path.join(config.IMAGE_FOLDER, image_name)
        print(f"--- Processing: {image_name} ---")
        extracted_text, latency = perform_ocr_and_save(image_path, config.OUTPUT_FOLDER)
        lines = extracted_text.split("\n") if extracted_text else []
        print(f"  → OCR completed in {latency:.2f}s | {len(lines)} lines extracted")
        if lines:
            print("  Preview (first 5 lines):")
            for i, line in enumerate(lines[:5]):
                print(f"   {i+1:2d}: {line}")
            if len(lines) > 5:
                print(f"   ... and {len(lines) - 5} more lines\n")

    print("\n✅ OCR extraction complete!")
    print(f"🗂️ Clean text files saved in: {config.OUTPUT_FOLDER}")

if __name__ == "__main__":
    main()


In [ ]:
# 1. Install Ollama python library and pandas
!pip install ollama pandas

# 2. Install Ollama System Binary
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess
import time

# Start ollama serve as a background process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("⏳ Starting Ollama server...")
time.sleep(10) # Give it a moment to start

print("⬇️ Pulling Model ")
!ollama pull gemma3:4b
print("✅ Setup Complete!")

In [ ]:
import os
import json
import re
import time
import ollama
from typing import Dict, Optional, Any, Tuple
from datetime import datetime

# --- Configuration ---
class Config:
    INPUT_TEXT_FOLDER = "ocr_texts"
    OUTPUT_FOLDER = "slm_json_results"
    MODEL_NAME = "gemma3:4b"
    LLM_TIMEOUT = 60

config = Config()

# --- VERSION 5: PROMPT WITH DATE SORTER SUPPORT ---
# --- VERSION 6: OPTIMIZED CHAIN-OF-THOUGHT PROMPT ---
SYSTEM_PROMPT = """
You are an expert Document Intelligence Agent specialized in Indian Driving Licenses. Your task is to extract structured data from raw OCR text with 100% precision.

### 1. EXTRACTION RULES (STRICT):

**A. NAME (CRITICAL):**
* **Goal:** Extract the License Holder's Name.
* **Logic:** The Holder's name is usually the **first** personal name appearing on the card, often below "DL No" or "Name".
* **NEGATIVE CONSTRAINT:** IGNORE any line starting with or containing: "S/o", "D/o", "W/o", "C/o", "Father", "Husband", "Son of", "Wife of".
* *Example:* If you see "Name: T VIJAYA" and "S/W/D: R SREEDHAR", extract "T VIJAYA".

**B. DATES (Contextual):**
* **Date of Birth (DOB):** Look for 'DOB', 'Birth', or the **oldest** date (e.g., 1970-2005).
* **Issue Date:** Look for 'DOI', 'Iss', 'Date of Issue'. Usually in the past.
* **Validity:** Look for 'Val', 'Exp', 'Valid Till'. Usually in the future (e.g., 2026-2045).
* *Format:* Standardize to DD-MM-YYYY.

**C. BLOOD GROUP (Anti-Hallucination):**
* **Rule:** ONLY extract if you clearly see a blood group pattern (A+, B+, O+, AB+, etc.).
* **Hallucination Check:** If the field is blank, missing, or contains "Unknown"/"None", return `null`.
* **Correction:** Treat '0+' (zero) as 'O+'. Treat 'B4' as 'B+'.

**D. ADDRESS:**
* Extract the full continuous address block. Ignore lines that are clearly headers (like "Union of India").

### 2. JSON OUTPUT FORMAT:
Return ONLY a valid JSON object. No markdown, no explanations.
{
    "name": "string or null",
    "blood_group": "string or null",
    "address": "string or null",
    "date_of_birth": "DD-MM-YYYY or null",
    "issue_date": "DD-MM-YYYY or null",
    "validity": "DD-MM-YYYY or null"
}
"""

# --- LLM Analyzer Class ---
class LLMAnalyzer:
    def __init__(self):
        self.client = ollama.Client()

    def clean_blood_group(self, bg: str) -> Optional[str]:
        """Fixes common OCR typos in Blood Group."""
        if not bg or str(bg).lower() == "null": return None
        bg = bg.upper().strip()
        bg = bg.replace('0', 'O').replace('Q', 'O').replace('4', '+').replace('VE', '')
        if re.match(r'^(A|B|AB|O)[+-]$', bg):
            return bg
        return None

    def parse_date(self, date_str: str) -> Optional[datetime]:
        """Helper to parse DD-MM-YYYY strings."""
        if not date_str: return None
        try:
            return datetime.strptime(date_str, "%d-%m-%Y")
        except ValueError:
            return None

    def sort_and_fix_dates(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """
        DATE SORTER LOGIC:
        1. Collects DOB, Issue Date, and Validity.
        2. Sorts them chronologically.
        3. Re-assigns based on logic: DOB (Oldest) < Issue (Middle) < Validity (Newest/Future).
        """

        # 1. Collect all valid dates found by the LLM
        found_dates = []
        for key in ["date_of_birth", "issue_date", "validity"]:
            d_str = data.get(key)
            d_obj = self.parse_date(d_str)
            if d_obj:
                found_dates.append(d_obj)

        # Remove duplicates and sort
        found_dates = sorted(list(set(found_dates)))

        count = len(found_dates)

        # 2. Apply Sorting Logic
        if count == 3:
            # Perfect Case: 3 dates found
            data["date_of_birth"] = found_dates[0].strftime("%d-%m-%Y") # Oldest
            data["issue_date"] = found_dates[1].strftime("%d-%m-%Y")    # Middle
            data["validity"] = found_dates[2].strftime("%d-%m-%Y")      # Newest

        elif count == 2:
            # Ambiguous Case: We have 2 dates. Usually DOB+Issue OR Issue+Validity.
            # Heuristic: Check if the newest date is far in the future (Validity)
            # or if the oldest date is very old (DOB).

            oldest = found_dates[0]
            newest = found_dates[1]
            today = datetime.now()

            # If newest is in the future, it is definitely Validity
            if newest > today:
                data["validity"] = newest.strftime("%d-%m-%Y")
                # The other date is likely Issue Date (unless it's very old, e.g. < 1980)
                if oldest.year < (today.year - 18):
                    # If > 18 years ago, safest to assume DOB (or an old Issue date).
                    # Let's verify against the LLM's original guess.
                    # For safety in this pipeline, we often assume Issue Date if it's recent enough.
                    data["issue_date"] = oldest.strftime("%d-%m-%Y")
                    data["date_of_birth"] = None # Reset potential error
                else:
                     data["issue_date"] = oldest.strftime("%d-%m-%Y")
            else:
                # Both dates are in the past. Likely DOB and Issue Date.
                data["date_of_birth"] = oldest.strftime("%d-%m-%Y")
                data["issue_date"] = newest.strftime("%d-%m-%Y")
                data["validity"] = None

        return data

    def analyze_with_llm(self, text: str) -> Tuple[Dict, float]:
        if not text.strip(): return {"error": "Empty text file"}, 0.0

        start_time = time.time()
        try:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"OCR Text:\n{text}"}
            ]

            response = self.client.chat(
                model=config.MODEL_NAME,
                messages=messages,
                options={'temperature': 0.01, 'num_ctx': 2048, 'timeout': config.LLM_TIMEOUT * 1000}
            )

            content = response['message']['content']
            json_result = self.extract_json_from_response(content)
            latency = time.time() - start_time

            return json_result, latency

        except Exception as e:
            print(f"❌ Error: {e}")
            return {"error": str(e)}, time.time() - start_time

    def extract_json_from_response(self, content: str) -> Dict[str, Any]:
        try:
            content = re.sub(r'```json\s*', '', content, flags=re.IGNORECASE)
            content = re.sub(r'```', '', content)
            start, end = content.find('{'), content.rfind('}') + 1
            if start != -1 and end != -1:
                content = content[start:end]

            result = json.loads(content)

            # --- Apply Cleaners & Sorters ---
            result["blood_group"] = self.clean_blood_group(result.get("blood_group"))
            result = self.sort_and_fix_dates(result) # <--- NEW DATE SORTER CALLED HERE

            # Sanitize nulls
            for k, v in result.items():
                if isinstance(v, str) and v.lower() == "null": result[k] = None

            # Filter final fields (we exclude DOB from final output if you don't want it,
            # but keeping it is useful for debugging. Here I stick to your original 5 fields).
            required_fields = ["name", "blood_group", "address", "issue_date", "validity"]
            return {k: result.get(k) for k in required_fields}

        except Exception as e:
            return {"error": f"Extraction error: {str(e)}"}

# --- Main Execution ---
def main():
    if not os.path.exists(config.INPUT_TEXT_FOLDER):
        os.makedirs(config.INPUT_TEXT_FOLDER)

    os.makedirs(config.OUTPUT_FOLDER, exist_ok=True)
    text_files = [f for f in os.listdir(config.INPUT_TEXT_FOLDER) if f.endswith('.txt')]

    print(f"🔎 Found {len(text_files)} files. Running Version 5 (With Date Sorter)...")
    analyzer = LLMAnalyzer()
    all_results = []

    for filename in text_files:
        filepath = os.path.join(config.INPUT_TEXT_FOLDER, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            raw_text = f.read()

        print(f"\n📄 Processing: {filename}")
        extracted_data, duration = analyzer.analyze_with_llm(raw_text)

        if "error" not in extracted_data:
            print(f"   ✅ Name: {extracted_data.get('name')}")
            print(f"   ✅ Issue: {extracted_data.get('issue_date')} | Valid: {extracted_data.get('validity')}")
        else:
            print(f"   ❌ Failed: {extracted_data['error']}")

        record = {"source_file": filename, "processing_time_seconds": round(duration, 2), **extracted_data}
        all_results.append(record)

    json_path = os.path.join(config.OUTPUT_FOLDER, "gemma3_4b_v5_sorted.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=4)

    print(f"\n🎉 Done! Results saved to {json_path}")

if __name__ == "__main__":
    main()